In [4]:
import numpy as np
import pandas as pd

In [2]:
df_a = pd.read_csv('dataset_a_original.csv', encoding = 'ISO-8859-1')

C:\Users\herna\anaconda3\envs\cygnus\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_a['Date'] = pd.to_datetime(df_a['Date'])

In [5]:
def cast_number(n):
    try:
        return float(n)
    except:
        return np.nan
df_a['WRank'] = df_a['WRank'].apply(lambda x: cast_number(x))
df_a['LRank'] = df_a['LRank'].apply(lambda x: cast_number(x))

In [6]:
df_a['Location'] = df_a['Location'].apply(lambda x: x.replace("'", ''))

In [1]:
df_a.to_csv(path_or_buf='dataset_1_parsed.csv', sep=',', index=False, header=False, )

NameError: name 'df_a' is not defined

In [20]:
df_b = pd.read_csv('dataset_b_original.csv')

In [97]:
def score_parser(score):
    """
    Sample inputs: 
        - 7-6(10) 7-6(1)
        - 12-10 7-5 4-6 7-5
    """
    
    # results container.
    parsed_score = {
        'winner_sets_won': 0,
        'loser_sets_won': 0,
        'winner_games_won': 0,
        'winner_games_lost': 0,
        'loser_games_won': 0,
        'loser_games_lost': 0
    }
    
    # split score by separator.
    score_list = score.split(' ')
    
    # iterate sets
    for index, match_set in enumerate(score_list):
        # check []
        if '[' in match_set or ']' or match_set:
            match_set = match_set.replace('[', '')
            match_set = match_set.replace(']', '')
        # get games for each player.
        games_a, games_b = match_set.split('-')
        # check for tiebreak.
        if '(' in games_b:
            # extract tiebreak score.
            tiebreak_score = int(games_b[games_b.index('(')+1 : games_b.index(')')])
            # update games_b string to remove tiebreak.
            games_b = games_b[0: games_b.index('(')]
        # cast all to int.
        games_a = int(games_a)
        games_b = int(games_b)
        # parse set stats.   
        parsed_score[f'winner_games_set_{index+1}'] = games_a
        parsed_score[f'loser_games_set_{index+1}'] = games_b   
        parsed_score['winner_games_won'] = parsed_score['winner_games_won'] + games_a
        parsed_score['loser_games_won'] = parsed_score['loser_games_won'] + games_b
        parsed_score['winner_games_lost'] = parsed_score['loser_games_won']
        parsed_score['loser_games_lost'] = parsed_score['winner_games_won']
        if games_a >= 6 or games_b >= 6:
            parsed_score['winner_sets_won'] = parsed_score['winner_sets_won'] + 1 if games_a > games_b else parsed_score['winner_sets_won']
            parsed_score['loser_sets_won'] = parsed_score['loser_sets_won'] + 1 if games_a < games_b else parsed_score['loser_sets_won']
            parsed_score['winner_sets_lost'] = parsed_score['loser_sets_won']
            parsed_score['loser_sets_lost'] = parsed_score['winner_sets_won']
    
    return parsed_score

for test in ['7-6(10) 7-6(1)', '12-10 7-5 4-6 7-5']:
    print(test)
    print(score_parser(test))

7-6(10) 7-6(1)
{'winner_sets_won': 2, 'loser_sets_won': 0, 'winner_games_won': 14, 'winner_games_lost': 12, 'loser_games_won': 12, 'loser_games_lost': 14, 'winner_games_set_1': 7, 'loser_games_set_1': 6, 'winner_sets_lost': 0, 'loser_sets_lost': 2, 'winner_games_set_2': 7, 'loser_games_set_2': 6}
12-10 7-5 4-6 7-5
{'winner_sets_won': 3, 'loser_sets_won': 1, 'winner_games_won': 30, 'winner_games_lost': 26, 'loser_games_won': 26, 'loser_games_lost': 30, 'winner_games_set_1': 12, 'loser_games_set_1': 10, 'winner_sets_lost': 1, 'loser_sets_lost': 3, 'winner_games_set_2': 7, 'loser_games_set_2': 5, 'winner_games_set_3': 4, 'loser_games_set_3': 6, 'winner_games_set_4': 7, 'loser_games_set_4': 5}


In [98]:
match_statistics = []

for index, row in df_b[['tourney_id', 'match_num', 'score']].iterrows():
    score = row[2]
    if not isinstance(score, str):
        continue
    if 'W/O' in score:
        continue
    if 'Walkover' in score:
        continue
    if 'In Progress' in score:
        continue
    if 'Played and unfinished' in score:
        continue
    if 'Default' in score:
        continue
    if 'Played and abandoned' in score:
        continue
    if 'Unfinished' in score:
        continue
    if 'RET' in score:
        continue
    if 'DEF' in score:
        continue
    if 'ABD' in score:
        continue
    if 'ABN' in score:
        continue
    # 
    try:
        parsed_match = score_parser(score.strip())
        parsed_match['tournament_id'] = row[0]
        parsed_match['match_num'] = row[1]
        match_statistics.append(parsed_match)
    except:
        continue

In [99]:
match_statistics_df = pd.DataFrame(match_statistics)

In [100]:
match_statistics_df[
    ['tournament_id', 'match_num', 
     'winner_games_set_1', 'loser_games_set_1', 
     'winner_games_set_2', 'loser_games_set_2', 
     'winner_games_set_3', 'loser_games_set_3', 
     'winner_games_set_4', 'loser_games_set_4', 
     'winner_games_set_5', 'loser_games_set_5',
     'winner_games_won', 'winner_games_lost',
     'loser_games_won', 'loser_games_lost',
     'winner_sets_won', 'winner_sets_lost',
     'loser_sets_won', 'loser_sets_lost'
    ]
].to_csv('match_statistics.csv', sep=',', index=False, header=False)

In [94]:
match_statistics_df.columns

Index(['winner_sets_won', 'loser_sets_won', 'winner_games_set_1',
       'loser_games_set_1', 'winner_sets_lost', 'loser_sets_lost',
       'winner_games_set_2', 'loser_games_set_2', 'winner_games_set_3',
       'loser_games_set_3', 'winner_games_set_4', 'loser_games_set_4',
       'tournament_id', 'match_num', 'winner_games_set_5',
       'loser_games_set_5'],
      dtype='object')

In [24]:
df_b['score'].unique
#match_statistics = []

<bound method Series.unique of 0         12-10 7-5 4-6 7-5
1               6-3 6-2 6-4
2           6-4 3-6 6-3 7-5
3               6-4 6-2 6-1
4               6-4 6-1 7-5
                ...        
169685          4-6 6-4 6-4
169686              6-1 6-1
169687       7-6(10) 7-6(1)
169688              6-4 6-4
169689              7-5 6-3
Name: score, Length: 169690, dtype: object>

In [8]:
df_b['tourney_name'] = df_b['tourney_name'].apply(lambda x: x.replace("'", ''))
df_b['tourney_date'] = df_b['tourney_date'].apply(lambda x: f'{str(x)[0:4]}-{str(x)[4:6]}-{str(x)[6:8]}')
df_b['tourney_date'] = pd.to_datetime(df_b['tourney_date'])
df_b['tourney_date']

0        1968-01-19
1        1968-01-19
2        1968-01-19
3        1968-01-19
4        1968-01-19
            ...    
169685   2019-02-25
169686   2019-02-25
169687   2019-02-25
169688   2019-02-25
169689   2019-02-25
Name: tourney_date, Length: 169690, dtype: datetime64[ns]

In [9]:
df_b.to_csv(path_or_buf='dataset_b_parsed.csv', sep=',', index=False, header=False, )